In [29]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt;
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [30]:
dfsongs = pd.read_csv("../train_triplets.txt", sep="\t", nrows=10000, names=['user', 'song', 'play_count'], header=None)

Let us inspect some of the imported records and other characteristics of data before we proceed further

In [31]:
dfsongs.head(5)

,user,song,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


In [32]:
from collections import defaultdict
usersDict = defaultdict(int)
with open("../train_triplets.txt", "r") as songFile:
    for record in songFile:
        #Fetch user id as the first item from a tab delimited line of record
        user = record.split("\t")[0]
        #Fetch song play count as the last item from a tab delimited line of record
        play_count = int(record.split("\t")[2])
        #Update song play count for a user
        usersDict[user] += play_count
        
userPlayCountList = [{'user': u, 'play_count': p} for u,p in usersDict.items()]
dfSongsPlayCount = pd.DataFrame(userPlayCountList)
#Arrange the user songs played records in descending order to facilitate further analysis
dfSongsPlayCount = dfSongsPlayCount.sort_values(by = 'play_count', ascending = False)
dfSongsPlayCount.to_csv("songs_play_count.csv", index=None)
dfSongsPlayCount = pd.read_csv("songs_play_count.csv")

In [33]:
totalPlayCount = sum(dfSongsPlayCount.play_count)
(float(dfSongsPlayCount.head(n=100000).play_count.sum())/totalPlayCount)*100
dfUserSubset = dfSongsPlayCount.head(n=100000)

In [34]:
songsDict = defaultdict(int)
with open("../train_triplets.txt", "r") as songFile:
    for record in songFile:
        #Fetch song id as the first item from a tab delimited line of record
        song = record.split("\t")[1]
        #Fetch song play count as the last item from a tab delimited line of record
        play_count = int(record.split("\t")[2])
        #Update song play count for a song
        songsDict[song] += play_count
        
songPlayCountList = [{'song': s, 'play_count': p} for s,p in songsDict.items()]
dfSongsPlaySummary = pd.DataFrame(songPlayCountList)
#Arrange the user songs played records in descending order to facilitate further analysis
dfSongsPlaySummary = dfSongsPlaySummary.sort_values(by = 'play_count', ascending = False)
dfSongsPlaySummary.to_csv("songs_summary.csv", index=None)

In [35]:
(float(dfSongsPlaySummary.head(n=30000).play_count.sum())/totalPlayCount)*100

78.39315366645269

In [36]:
dfSongPlayCountSubset = dfSongsPlaySummary.head(n=30000)

In [37]:
dfAllSongs = pd.read_csv("../train_triplets.txt", sep="\t", names=['user', 'song', 'play_count'], header=None)
#Filter records for top 40% users using the users subset found earlier
dfSongsTopUsers = dfAllSongs[dfAllSongs.user.isin(list(dfUserSubset.user))]
dfSongsTopSongs = dfSongsTopUsers[dfSongsTopUsers.song.isin(list(dfSongPlayCountSubset.song))]

In [38]:
#Save the top 30 songs dataset to a file
dfSongsTopSongs.to_csv("top_Songs_Subset.csv", index=False)
#Find the number of records in this subset
dfSongsTopSongs.shape

(10774558, 3)

In [39]:
del dfAllSongs
del dfSongsTopUsers

In [40]:
dfSongsTopSongs.head(5)

,user,song,play_count
498,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOADQPP12A67020C82,12
499,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAFTRR12AF72A8D4D,1
500,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOANQFY12AB0183239,1
501,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAYATB12A6701FD50,1
502,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBOAFP12A8C131F36,7


In [41]:
import sqlite3
conn = sqlite3.connect('.././MillionSongSubset/track_metadata.db')
cur = conn.cursor()
#Find the name of tables in the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
cur.fetchall()

[('songs',)]

In [42]:
#Retrieve details of song tracks from the metadata file
dfTrackMetadata = pd.read_sql(con=conn, sql='select * from songs')
dfTrackMetadata_sub = dfTrackMetadata[dfTrackMetadata.song_id.isin(list(dfSongPlayCountSubset.song))]

In [43]:
dfTrackMetadata_sub.head(3)

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
115,TRMMGCB128E079651D,Get Along (Feat: Pace Won) (Instrumental),SOHNWIM12A67ADF7D9,Charango,ARU3C671187FB3F71B,067102ea-9519-4622-9077-57ca4164cfbb,Morcheeba,227.47383,0.819087,0.533117,2002,185967,-1,0
123,TRMMGTX128F92FB4D9,Viejo,SOECFIW12A8C144546,Caraluna,ARPAAPH1187FB3601B,f69d655c-ffd6-4bee-8c2a-3086b2be2fc6,Bacilos,307.51302,0.595554,0.400705,0,6825058,-1,0
145,TRMMGDP128F933E59A,I Say A Little Prayer,SOGWEOB12AB018A4D0,The Legendary Hi Records Albums_ Volume 3: Ful...,ARNNRN31187B9AE7B7,fb7272ba-f130-4f0a-934d-6eeea4c18c9a,Al Green,133.58975,0.779490,0.599210,1978,5211723,-1,11898


In [44]:
del(dfTrackMetadata_sub['track_id'])
del(dfTrackMetadata_sub['artist_mbid'])
#Drop duplicate songs ids, as we are concerned with only unique songs
dfTrackMetadata_sub.drop_duplicates(['song_id'], inplace=True)
#Join the earlier obtained top popular songs dataset and this metadata dataframe on song id
dfPopularSongMetaDataMerged = pd.merge(dfSongsTopSongs, dfTrackMetadata_sub, how="left", 
                                      left_on="song", right_on = "song_id")
#Keep only relevant columns
dfPopularSongMetaDataMerged = dfPopularSongMetaDataMerged[['user', 'song', 'title',
                                                           'play_count', 'release', 'artist_name', 'year']]
dfPopularSongMetaDataMerged.head(3)

C:\Users\soni2\AppData\Local\Temp\ipykernel_18780\3474166419.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTrackMetadata_sub.drop_duplicates(['song_id'], inplace=True)


,user,song,title,play_count,release,artist_name,year
0,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOADQPP12A67020C82,You And Me Jesus,12,Tribute To Jake Hess,Jake Hess,2004
1,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAFTRR12AF72A8D4D,Harder Better Faster Stronger,1,Discovery,Daft Punk,2007
2,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOANQFY12AB0183239,Uprising,1,Uprising,Muse,0


In [45]:
def create_popularity_recommendation(songData, user, song):
    #Get a count of users for each unique song as recommendation score
    songDataGrouped = songData.groupby([song]).agg({user: 'count'}).reset_index()
    songDataGrouped.rename(columns = {user: 'score'},inplace=True)
    
    #Sort the songs based upon recommendation score
    songDataSorted = songDataGrouped.sort_values(['score', song], ascending = [0,1])
    
    #Generate a recommendation rank based upon score
    songDataSorted['Rank'] = songDataSorted['score'].rank(ascending=0, method='first')
        
    #Get the top 10 recommendations
    popularityBasedRecommendations = songDataSorted.head(10)
    return popularityBasedRecommendations

In [46]:
recommendations = create_popularity_recommendation(dfPopularSongMetaDataMerged,'user','title')
recommendations

,title,score,Rank
19580,Sehr kosmisch,18626,1.0
5780,Dog Days Are Over (Radio Edit),17635,2.0
27314,You're The One,16085,3.0
19542,Secrets,15138,4.0
18636,Revelry,14945,5.0
25070,Undo,14687,6.0
7530,Fireflies,13085,7.0
9640,Hey_ Soul Sister,12993,8.0
25216,Use Somebody,12793,9.0
9921,Horn Concerto No. 4 in E flat K495: II. Romanc...,12346,10.0


In [47]:
#Aggregate by user and calculate sum of all songs listened by an user
dfPopularSongMetaDataMergedSum  = dfPopularSongMetaDataMerged[['user','play_count']].groupby('user').sum().reset_index()
dfPopularSongMetaDataMergedSum.rename(columns={'play_count':'total_play_count'},inplace=True)
dfPopularSongMetaDataMerged = pd.merge(dfPopularSongMetaDataMerged,dfPopularSongMetaDataMergedSum)
#Calculate the fractional play count
dfPopularSongMetaDataMerged['fractional_play_count'] = dfPopularSongMetaDataMerged['play_count']/dfPopularSongMetaDataMerged['total_play_count']

In [48]:
#Examine the structure of the newly formed dataset
dfPopularSongMetaDataMerged.head()

,user,song,title,play_count,release,artist_name,year,total_play_count,fractional_play_count
0,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOADQPP12A67020C82,You And Me Jesus,12,Tribute To Jake Hess,Jake Hess,2004,329,0.036474
1,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAFTRR12AF72A8D4D,Harder Better Faster Stronger,1,Discovery,Daft Punk,2007,329,0.003040
2,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOANQFY12AB0183239,Uprising,1,Uprising,Muse,0,329,0.003040
3,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAYATB12A6701FD50,Breakfast At Tiffany's,1,Home,Deep Blue Something,1993,329,0.003040
4,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBOAFP12A8C131F36,Lucky (Album Version),7,We Sing. We Dance. We Steal Things.,Jason Mraz & Colbie Caillat,0,329,0.021277


In [49]:
from scipy.sparse import coo_matrix

small_set = dfPopularSongMetaDataMerged

#Convert user and song IDs to integer values that can be handled by numpy matices
user_codes = small_set.user.drop_duplicates().reset_index()
song_codes = small_set.song.drop_duplicates().reset_index()
user_codes.rename(columns={'index':'user_index'}, inplace=True)
song_codes.rename(columns={'index':'song_index'}, inplace=True)
song_codes['so_index_value'] = list(song_codes.index)
user_codes['us_index_value'] = list(user_codes.index)
small_set = pd.merge(small_set,song_codes,how='left')
small_set = pd.merge(small_set,user_codes,how='left')
mat_candidate = small_set[['us_index_value','so_index_value','fractional_play_count']]
data_array = mat_candidate.fractional_play_count.values
row_array = mat_candidate.us_index_value.values
col_array = mat_candidate.so_index_value.values

#Create a sparse matrix
data_sparse = coo_matrix((data_array, (row_array, col_array)),dtype=float)
data_sparse

<99996x30000 sparse matrix of type '<class 'numpy.float64'>'
	with 10774558 stored elements in COOrdinate format>

In [50]:
user_codes[user_codes.user =='2a2f776cbac6df64d6cb505e7e834e01684673b6']

,user_index,user,us_index_value
27516,2981434,2a2f776cbac6df64d6cb505e7e834e01684673b6,27516


In [51]:
import importlib
import SVD_Helper
importlib.reload(SVD_Helper)

<module 'SVD_Helper' from 'c:\\Users\\soni2\\Desktop\\BITSH\\AI\\AI_Group30\\MatrixFactorization\\SVD_Helper.py'>

In [54]:
import SVD_Helper

K=2
utilityMatrix = data_sparse
MAX_PID = utilityMatrix.shape[1]
MAX_UID = utilityMatrix.shape[0]

U, S, Vt = SVD_Helper.compute_svd(utilityMatrix, K)

In [55]:
userid = input("Enter user ID: ")
uTest = [int(userid)]

uTest_recommended_items = SVD_Helper.compute_estimated_matrix(utilityMatrix, U, S, Vt, uTest, K, MAX_PID, MAX_UID, True)

for  user  in  uTest:
    print("\n-------------------------------------- \
    \nRecommendation for user with id: {}\n--------------------------------------\n". format(user))
    rank_value = 1
    for i in uTest_recommended_items[user,0:10]:
        song_details = small_set[small_set.so_index_value == i].drop_duplicates('so_index_value')[['title','artist_name']]
        print("#{}: {} By {}".format(rank_value, list(song_details['title'])[0],list(song_details['artist_name'])[0]))
        rank_value+=1

MemoryError: Unable to allocate 5.59 GiB for an array with shape (99996, 30000) and data type float16